<a href="https://colab.research.google.com/github/quantumseminar/textbook/blob/main/02qa/qa5_tytan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## QUBOアニーリングツールを活用したプログラミング
前回制約条件とコスト関数から問題を定式化しました。この式を実際のPythonプログラミングに落とし込んで解くことができます。

## 問題
2台の自動車が同じスタート地点からゴール地点までいきます。１台の車には3つのルートが提案されるものとします。なるべく複数の車が同じ道を通らないようにルートの組合せを選んでください。

![../img/qa4_01.jpg](https://github.com/quantumseminar/textbook/blob/main/img/qa4_01.jpg?raw=1)

とりうるルートの候補は、

car1,car2ともに、

route1: s0 > s1 > s4 > s9   
route2: s0 > s3 > s8 > s11   
route3: s2 > s7 > s10 > s11

とする。

## 定式化
制約条件は制約条件でまとめる。

$$
H_{const} = H_0 + H_1 = (q_0 + q_1 + q_2 - 1)^2 +  (q_3 + q_4 + q_5 - 1)^2
$$

コスト関数は道の混雑度を全部足して、

$$
H_{cost} = (q_0 + q_1 + q_3 + q_4)^2 + (q_0 + q_3)^2 + (q_2 + q_5)^2 + (q_1 + q_4)^2 + (q_0 + q_3)^2 + (q_2 + q_5)^2 + (q_1 + q_4)^2 + (q_0 + q_3)^2 +  (q_2 + q_5)^2 +  (q_1 + q_2 + q_4 + q_5)^2
$$

全部足すときに調整変数を挟む必要がある。

$$
H = H_{const} + M* H_{cost}
$$

## Tytanを使った計算
定式化をプログラムの形にするためにライブラリ（追加のアプリみたいな便利なツール）を利用します。
まずはインストールします。Google Colabを使う時は毎回インストールが必要です。

In [ ]:
!pip install tytan

## ツールを読み込む
インストールしたライブラリを読み込みます。今回読み込みは一行で終わりです。

In [2]:
from tytan import *

## 量子ビットを決める
今回は6量子ビット使いますので、6個指定します。

In [3]:
#量子ビットを用意
q = symbols_list(6, 'q{}')

## 式を入力する（制約条件）
工夫しても入れられますが、今回は地道に入れます。まずは制約条件。

$$
H_{const} = H_0 + H_1 = (q_0 + q_1 + q_2 - 1)^2 +  (q_3 + q_4 + q_5 - 1)^2
$$


In [4]:
Hconst = (q[0]+q[1]+q[2]-1)**2 + (q[3]+q[4]+q[5]-1)**2

## 式を入力する（コスト関数）
次にコスト関数

$$
H_{cost} = (q_0 + q_1 + q_3 + q_4)^2 + (q_0 + q_3)^2 + (q_2 + q_5)^2 + (q_1 + q_4)^2 + (q_0 + q_3)^2 + (q_2 + q_5)^2 + (q_1 + q_4)^2 + (q_0 + q_3)^2 +  (q_2 + q_5)^2 +  (q_1 + q_2 + q_4 + q_5)^2
$$

In [5]:
Hcost = (q[0]+q[1]+q[3]+q[4])**2 + (q[0]+q[3])**2 + (q[2]+q[5])**2 + (q[1]+q[4])**2 + (q[0]+q[3])**2 + (q[2]+q[5])**2 + (q[1]+q[4])**2 + (q[0]+q[3])**2 + (q[2]+q[5])**2 + (q[1]+q[2]+q[4]+q[5])**2

## 二つの式を繋げる
繋げる時は調整変数（ちょうせいへんすう）と言って二つの式のバランスを決める値を挟む必要がありました。Mを設定してみます。


In [6]:
M = 1
H = Hcost + M*Hconst

式を足し合わせた結果を見てみます。

In [7]:
print(H)

3*(q0 + q3)**2 + 2*(q1 + q4)**2 + 3*(q2 + q5)**2 + (-1 + q0 + q1 + q2)**2 + (-1 + q3 + q4 + q5)**2 + (q0 + q1 + q3 + q4)**2 + (q1 + q2 + q4 + q5)**2


展開はされていませんが、式は統合されたようです。

## コンパイル
今回は式を量子コンピュータや量子アニーラが読める形に変形します。コンパイルというのを使います。
quboは式を縦横に数字が並んだ行列の形に。offsetは計算に関係のない固定の数字を抜き出します。

In [8]:
#コンパイル
qubo, offset = Compile(H).get_qubo()

In [9]:
print(qubo)

{('q0', 'q0'): 3.0, ('q0', 'q4'): 2.0, ('q3', 'q4'): 4.0, ('q1', 'q1'): 3.0, ('q1', 'q2'): 4.0, ('q4', 'q4'): 3.0, ('q0', 'q3'): 8.0, ('q0', 'q1'): 4.0, ('q4', 'q2'): 2.0, ('q2', 'q2'): 3.0, ('q5', 'q2'): 8.0, ('q5', 'q4'): 4.0, ('q0', 'q2'): 2.0, ('q3', 'q3'): 3.0, ('q5', 'q5'): 3.0, ('q5', 'q1'): 2.0, ('q5', 'q3'): 2.0, ('q4', 'q1'): 8.0, ('q3', 'q1'): 2.0}


In [10]:
print(offset)

2.0


あまり気にしないで、作業として計算をさせます。実はこのような計算を解いてくれるプログラムをソルバーとかサンプラーと呼んでいます。いろんな用途などに対応して色々なサンプラーがあります。今回はArminSamplerを使います。


In [11]:
#サンプラー選択
solver = sampler.ArminSampler()

次に実際の計算を実行します。式を作ってサンプラーを選んだら実行するのみです。こうした計算の答えは決まった一つの答えが出るというよりも、いくつかの答えが出てくるのでサンプリングとも呼ばれます。

In [12]:
#サンプリング
result = solver.run(qubo)

MODE: CPU
DEVICE: cpu


In [13]:
#結果
for r in result:
    print(r)

[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 0.0}, 0.0, 100]


計算結果は100回計算してエネルギー0で全部0という答えが出ましたが、これはどの道も選んでないことになります。制約条件が満たされていないので、Mの値を大きくしてみます。

In [17]:
M = 5
H = Hcost + M*Hconst

#コンパイル
qubo, offset = Compile(H).get_qubo()

#サンプリング
result = solver.run(qubo)

#結果
for r in result:
    print(r)

MODE: CPU
DEVICE: cpu
[{'q0': 0.0, 'q1': 0.0, 'q2': 1.0, 'q3': 1.0, 'q4': 0.0, 'q5': 0.0}, -2.0, 45]
[{'q0': 1.0, 'q1': 0.0, 'q2': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 1.0}, -2.0, 38]
[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 0.0, 'q4': 1.0, 'q5': 0.0}, -1.0, 14]
[{'q0': 0.0, 'q1': 1.0, 'q2': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 0.0}, -1.0, 3]


今回は答えが45回エネルギーが-2.0の答えが出て、38回エネルギーが-2.0の答えが出ました。中を見てみると、

上の答えは、
car1はroute3を、car2はroute1を通るという答え、

下の答えは、
car1はroute1を、car3はroute3を通るという答えになりました。

route1: s0 > s1 > s4 > s9   
route2: s0 > s3 > s8 > s11   
route3: s2 > s7 > s10 > s11

![../img/qa4_01.jpg](https://github.com/quantumseminar/textbook/blob/main/img/qa4_01.jpg?raw=1)


